In [1]:
import pyodbc
import distance
import jieba
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import synonyms
from gensim.models import *
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pandas import DataFrame
import pandas as pd
import time

#連線local DB
# cnxn = pyodbc.connect(r'Driver=SQL Server;Server=LAPTOP-VFEIS9GU;Database=DIET;Trusted_Connection=yes;') 
# cursor = cnxn.cursor()
# cursor.execute("SELECT Name FROM Eat_out")

#連線74
conn = pyodbc.connect(r'DRIVER={SQL Server};SERVER=10.40.8.35;PORT=1443;DATABASE=NutritionSurvey;UID=sa;PWD=1qaz2wsx3EDC;charset=utf8')
cursor=conn.cursor() 
cursor.execute("SELECT Name FROM Recipe")

corpus_data = []
result = cursor.fetchall()  #逐項讀DB
for row in result:
    corpus_data.append(row)

#字串處理
corpus_data=str(corpus_data).replace('(','').replace(')','').replace(',, ',',').replace('[','').replace(']','').replace("'","").replace(' ','')  
# corpus_data=str(corpus_data).replace(',, ',',').replace('[','').replace(']','').replace("'","").replace(' ','')  

corpus_data=corpus_data.split(',')
corpus_data=list(corpus_data)

while '' in corpus_data:
    corpus_data.remove('')

# 移除空格行
# k=1
# for i in range(len(corpus_data)):
#     if i%2==1:
#         del corpus_data[k]
#         k=k+1

[jieba] default dict file path ..\data\vocab.txt


Building prefix dict from C:\Users\HANK\anaconda3\envs\BERT\lib\site-packages\synonyms\data\vocab.txt ...
Loading model from cache C:\Users\HANK\AppData\Local\Temp\jieba.ufe45f15788576ee60c7b833f2eb7e074.cache


>> Synonyms load wordseg dict [C:\Users\HANK\anaconda3\envs\BERT\lib\site-packages\synonyms\data\vocab.txt] ... 


Loading model cost 1.110 seconds.
Prefix dict has been built successfully.


>> Synonyms on loading stopwords [C:\Users\HANK\anaconda3\envs\BERT\lib\site-packages\synonyms\data\stopwords.txt] ...
[Synonyms] on loading vectors [C:\Users\HANK\anaconda3\envs\BERT\lib\site-packages\synonyms\data\words.vector.gz] ...


C:\Users\HANK\anaconda3\envs\BERT\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\HANK\anaconda3\envs\BERT\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\HANK\anaconda3\envs\BERT\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\HANK\anaconda3\envs\BERT\lib\site-packages\tensorflow\python\framework

In [2]:
def Levenshtein(keyword):
    def edit_distance(corpus_data, keyword):
        return distance.levenshtein(corpus_data, keyword)
    i=0
    for i in range(10):
        i=i+1
        results = list(filter(lambda x: edit_distance(x, keyword) <= i, corpus_data))
        if(len(results)>9):
#             print("step",i,results)
            break
        else:
#             print("step",i,"未達top10")
            continue

    long=len(results)           #create等長list (此算法無相似值結果,故創空欄位合併用)
    Similarity=['-']*long           

    #df sort
    dict_df1={
             "Similarity":Similarity,   
             "Name":results
            }
    data1=DataFrame(dict_df1)
    sult1=data1
    return(sult1)

def BM25(keyword):
    tokenized_corpus = [list(jieba.cut(doc)) for doc in corpus_data]
    bm25 = BM25Okapi(tokenized_corpus)

    tokenized_query = list(jieba.cut(keyword))
    doc_scores = bm25.get_scores(tokenized_query)
    top10 = bm25.get_top_n(tokenized_query,corpus_data,n=10)

    # print("斷詞結果:",tokenized_corpus)

    list_doc_scores = list(doc_scores)
    list_corpus = list(corpus_data)

    #df sort
    dict_df2={"Similarity":list_doc_scores,
             "Name":list_corpus
            }
    data2=DataFrame(dict_df2)
    sult2=data2.sort_values(by=['Similarity'],ascending=False)
    sult2 = sult2.reset_index(drop=True) #reset df index  
    return(sult2)

def jaccard(keyword):
    def jaccard_similarity(corpus_data,keyword):
        def add_space(s):
            return ' '.join(list(s))

        # 將字中間加入空格
        corpus_data, keyword = add_space(corpus_data), add_space(keyword)
        # 轉化為TF矩陣
        cv = CountVectorizer(tokenizer=lambda s: s.split())
        corpus = [corpus_data, keyword]
        vectors = cv.fit_transform(corpus).toarray()
        # 求交集
        numerator = np.sum(np.min(vectors, axis=0))
        # 求聯集
        denominator = np.sum(np.max(vectors, axis=0))
        # 計算傑卡德係數
        return 1.0 * numerator / denominator

    list_name=[]
    list_similarity=[]
    i=-1
    for abc in corpus_data:
        i=i+1
        Similarity=jaccard_similarity(corpus_data[i], keyword)
        list_name.append(corpus_data[i])
        list_similarity.append(Similarity)

    #df sort
    dict_df3={"Similarity":list_similarity,
              "Name":list_name
               }
    data3=DataFrame(dict_df3)
    sult3=data3.sort_values(by=['Similarity'],ascending=False)
    sult3 = sult3.reset_index(drop=True) #reset df index
    return(sult3)

def TFIDF(keyword):
    class TF_IDF_Model(object):
        def __init__(self, corpus_data):
            self.corpus_data = corpus_data
            self.documents_number = len(corpus_data)
            self.tf = []
            self.idf = {}
            self.init()

        def init(self):
            df = {}
            for document in self.corpus_data:
                temp = {}
                for word in document:
                    temp[word] = temp.get(word, 0) + 1/len(document)
                self.tf.append(temp)
                for key in temp.keys():
                    df[key] = df.get(key, 0) + 1
            for key, value in df.items():
                self.idf[key] = np.log(self.documents_number / (value + 1))

        def get_score(self, index, keyword):
            score = 0.0
            for q in keyword:
                if q not in self.tf[index]:
                    continue
                score += self.tf[index][q] * self.idf[q]
            return score

        def get_documents_score(self, keyword):
            score_list = []
            for i in range(self.documents_number):
                score_list.append(self.get_score(i, keyword))
            return score_list
        
    tf_idf_model = TF_IDF_Model(corpus_data)
    scores = tf_idf_model.get_documents_score(keyword)

    #df sort
    dict_df4={"Similarity":scores,
             "Name":corpus_data
            }
    data4=DataFrame(dict_df4)
    sult4=data4.sort_values(by=['Similarity'],ascending=False)
    sult4 = sult4.reset_index(drop=True) #reset df index
    return(sult4)

def Synonyms(keyword):
    Similarity_list=[]
    name_list=[]

    i=-1
    for abc in corpus_data:
        i=i+1
        r = synonyms.compare(corpus_data[i], keyword, seg=False)
        Similarity_list.append(r)
        name_list.append(corpus_data[i])

    dict_df5={"Similarity":Similarity_list,
             "Name":name_list
            }
    data5=DataFrame(dict_df5)
    sult5=data5.sort_values(by=['Similarity'],ascending=False)
    sult5 = sult5.reset_index(drop=True) #reset df index
    return(sult5)

def Algorithm5_Vote_Top1(keyword):
    np.seterr(divide='ignore',invalid='ignore')
    df_top1 = pd.concat([Levenshtein(keyword)[0:1],jaccard(keyword)[0:1],BM25(keyword)[0:1],TFIDF(keyword)[0:1],Synonyms(keyword)[0:1]],axis=0)
    freq_top1 = df_top1.groupby(['Name']).count() 
    freq_top1 = freq_top1.sort_values(by=['Similarity'],ascending=False) #sort
    return(freq_top1)

def Algorithm5_Vote_Top10(keyword):
    np.seterr(divide='ignore',invalid='ignore')
    df_top10 = pd.concat([Levenshtein(keyword)[0:10],jaccard(keyword)[0:10],BM25(keyword)[0:10],TFIDF(keyword)[0:10],Synonyms(keyword)[0:10]],axis=0)
    freq_top10 = df_top10.groupby(['Name']).count()
    freq_top10 = freq_top10.sort_values(by=['Similarity'],ascending=False) #sort
    return(freq_top10)

In [5]:
%%time
Algorithm5_Vote_Top10("提拉米速")

Wall time: 1.04 s


,Similarity
Name,
提拉米蘇,4
爆米花,3
炒米粉,3
米蛋餅,2
米苔目,2
米月餅,2
筒仔米糕,2
玉米蛋沙拉土司,2
溫沙拉,2


In [78]:
%%time
Algorithm5_Vote_Top10("牛鞭湯")

Wall time: 982 ms


,Similarity
Name,
番茄牛肉湯,3
清牛肉湯麵,3
牛軋糖,3
牛角包,3
牛排餐,2
煎牛排,2
生吐司,2
韓式牛肉清湯,2
蔥油餅,2
